In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pickle
import os
import base64
from os import listdir
import csv
import matplotlib.pyplot as plt
import cv2
import numpy as np
import matplotlib.pyplot as plt


In [4]:
file_path = '/content/drive/My Drive/final_year/profile_dict.pickle'


with open(file_path, 'rb') as f:
    encoded_data = f.read()

# Decode the Base64-encoded data
decoded_bytes = base64.b64decode(encoded_data)

# Deserialize the object using pickle
img_dict = pickle.loads(decoded_bytes)
print(img_dict.keys())


dict_keys(['x', 'y'])


In [5]:
import numpy as np
from scipy.ndimage import zoom

x_resized = []
y_resized = []

for i in img_dict['x']:
    resized_x = zoom(i, (224/i.shape[0], 224/i.shape[1], 1), order=1)
    x_resized.append(resized_x)

for i in img_dict['y']:

    y_resized.append(i)

# Convert lists to numpy arrays
x_resized = np.array(x_resized)
y_resized = np.array(y_resized)

print("Resized x shape:", x_resized.shape)
print("Resized y shape:", y_resized.shape)


Resized x shape: (925, 224, 224, 3)
Resized y shape: (925, 5)


In [6]:
y_resized=y_resized.astype(float)
x_data=x_resized
y_data=y_resized

In [7]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,Dropout,MaxPooling2D,Dense,Input,Reshape, BatchNormalization,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.losses import MeanSquaredError
import math
from tensorflow.keras.optimizers import Adam

In [8]:
from tensorflow.keras.applications import VGG16

# Load the pre-trained VGG16 model without the fully connected layers
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg_base.trainable=False
vgg_base.summary()


58889256/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [9]:

# Add custom fully connected layers
x = Flatten()(vgg_base.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(16, activation='relu')(x)
output = Dense(5)(x)

# Create the model
classifier = Model(inputs=vgg_base.input, outputs=output)

# Compile the model
loss = MeanSquaredError()
classifier.compile(optimizer=Adam(learning_rate=0.0001), loss=loss, metrics=[tf.keras.metrics.MeanSquaredError()])

# classifier.compile(optimizer='adam', loss=loss, metrics=[tf.keras.metrics.MeanSquaredError()])

classifier.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [10]:
from tensorflow.keras.callbacks import ReduceLROnPlateau


# Define the ReduceLROnPlateau callback
reduce_lr = ReduceLROnPlateau(factor=0.3, patience=5, verbose=1)

# Train the model with the callback
history = classifier.fit(x_data, y_data, epochs=40, batch_size=64, callbacks=[reduce_lr], validation_split=0.2)


Epoch 1/40
12/12 [==============================] - 49s 3s/step - loss: 194.0041 - mean_squared_error: 194.0041 - val_loss: 3.6144 - val_mean_squared_error: 3.6144 - lr: 1.0000e-04
Epoch 2/40
12/12 [==============================] - 4s 328ms/step - loss: 42.1602 - mean_squared_error: 42.1602 - val_loss: 1.4624 - val_mean_squared_error: 1.4624 - lr: 1.0000e-04
Epoch 3/40
12/12 [==============================] - 4s 373ms/step - loss: 14.4581 - mean_squared_error: 14.4581 - val_loss: 0.6542 - val_mean_squared_error: 0.6542 - lr: 1.0000e-04
Epoch 4/40
12/12 [==============================] - 4s 377ms/step - loss: 7.0101 - mean_squared_error: 7.0101 - val_loss: 0.3073 - val_mean_squared_error: 0.3073 - lr: 1.0000e-04
Epoch 5/40
12/12 [==============================] - 4s 378ms/step - loss: 2.3623 - mean_squared_error: 2.3623 - val_loss: 0.2046 - val_mean_squared_error: 0.2046 - lr: 1.0000e-04
Epoch 6/40
12/12 [==============================] - 4s 374ms/step - loss: 1.8300 - mean_squared_err

In [11]:
# Save model architecture, weights, and optimizer state
classifier.save('/content/drive/My Drive/final_year/profile_model_vgg_tl.keras')

# Save training history
np.save('/content/drive/My Drive/final_year/profile_history_vgg_tl.npy', history.history)


In [12]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

# Load the pre-trained VGG16 model
vgg_base1 = VGG16(weights='imagenet', include_top=True)

# Remove the last layer (prediction layer) from the model
vgg_base_truncated = Model(inputs=vgg_base1.input, outputs=vgg_base1.layers[-2].output)


vgg_base_truncated.summary()


553467096/553467096 [==============================] - 11s 0us/step
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                         

In [14]:
vgg_base_truncated.trainable = False

for layer in vgg_base_truncated.layers[-7:]:
    layer.trainable = True
for layer in vgg_base_truncated.layers:
  print(layer,"    ",layer.trainable)

<keras.src.engine.input_layer.InputLayer object at 0x7c0c011af8b0>      False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7c0bf01e7d30>      False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7c0ce5515cf0>      False
<keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7c0bf01e7f70>      False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7c0bf01e72b0>      False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7c0bf01e7130>      False
<keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7c0bf0222680>      False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7c0c0119fa00>      False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7c0bf02229e0>      False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7c0bf0222ad0>      False
<keras.src.layers.pooling.max_pooling2d.MaxPooling2D object at 0x7c0bf0223910>      False
<keras.src.layers.convolutional.conv2d.Conv2D object at 0x7c0bf02229b0>

In [15]:
# Add custom fully connected layers
x = (vgg_base_truncated.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(32, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(16, activation='relu')(x)
output = Dense(5)(x)

# Create the model
classifier1 = Model(inputs=vgg_base_truncated.input, outputs=output)

# Compile the model
loss = MeanSquaredError()
classifier1.compile(optimizer=Adam(learning_rate=0.0001), loss=loss, metrics=[tf.keras.metrics.MeanSquaredError()])

# classifier.compile(optimizer='adam', loss=loss, metrics=[tf.keras.metrics.MeanSquaredError()])

classifier1.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [17]:
from tensorflow.keras.callbacks import ReduceLROnPlateau


# Define the ReduceLROnPlateau callback
reduce_lr = ReduceLROnPlateau(factor=0.3, patience=5, verbose=1)

# Train the model with the callback
history1 = classifier1.fit(x_data, y_data, epochs=20, batch_size=64, callbacks=[reduce_lr], validation_split=0.2)


Epoch 1/20
12/12 [==============================] - 5s 424ms/step - loss: 0.0116 - mean_squared_error: 0.0116 - val_loss: 0.0096 - val_mean_squared_error: 0.0096 - lr: 1.0000e-04
Epoch 2/20
12/12 [==============================] - 5s 426ms/step - loss: 0.0112 - mean_squared_error: 0.0112 - val_loss: 0.0089 - val_mean_squared_error: 0.0089 - lr: 1.0000e-04
Epoch 3/20
12/12 [==============================] - 6s 472ms/step - loss: 0.0104 - mean_squared_error: 0.0104 - val_loss: 0.0084 - val_mean_squared_error: 0.0084 - lr: 1.0000e-04
Epoch 4/20
12/12 [==============================] - 5s 425ms/step - loss: 0.0106 - mean_squared_error: 0.0106 - val_loss: 0.0075 - val_mean_squared_error: 0.0075 - lr: 1.0000e-04
Epoch 5/20
12/12 [==============================] - 5s 455ms/step - loss: 0.0101 - mean_squared_error: 0.0101 - val_loss: 0.0077 - val_mean_squared_error: 0.0077 - lr: 1.0000e-04
Epoch 6/20
12/12 [==============================] - 5s 406ms/step - loss: 0.0097 - mean_squared_error: 0.

In [18]:
# Save model architecture, weights, and optimizer state
classifier1.save('/content/drive/My Drive/final_year/profile_model_vgg.keras')

# Save training history
np.save('/content/drive/My Drive/final_year/profile_history_vgg.npy', history1.history)


In [19]:
x_train=x_resized
y_train=y_resized

In [20]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

val_preds=classifier1.predict(x_train)

mse=history1.history['loss'][-1]
val_mse=history1.history['val_loss'][-1]

mae=mean_absolute_error(y_train,val_preds)

rmse=np.sqrt(val_mse)
r2 = r2_score(y_train, val_preds)

# Print the calculated metrics
print("Mean Squared Error (MSE):", mse)
print("Validation Mean Squared Error (val_MSE):", val_mse)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R^2 score:", r2)

29/29 [==============================] - 19s 392ms/step
Mean Squared Error (MSE): 0.005944403354078531
Validation Mean Squared Error (val_MSE): 0.004126373678445816
Mean Absolute Error (MAE): 0.05187957541220907
Root Mean Squared Error (RMSE): 0.06423685607535456
R^2 score: -3.3663217825482183


In [21]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

val_preds=classifier.predict(x_train)

mse=history.history['loss'][-1]
val_mse=history.history['val_loss'][-1]

mae=mean_absolute_error(y_train,val_preds)

rmse=np.sqrt(val_mse)
r2 = r2_score(y_train, val_preds)

# Print the calculated metrics
print("Mean Squared Error (MSE):", mse)
print("Validation Mean Squared Error (val_MSE):", val_mse)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("R^2 score:", r2)

29/29 [==============================] - 4s 130ms/step
Mean Squared Error (MSE): 0.04005306586623192
Validation Mean Squared Error (val_MSE): 0.09802692383527756
Mean Absolute Error (MAE): 0.20382125843346843
Root Mean Squared Error (RMSE): 0.3130925164153202
R^2 score: -72.52840189190607


In [22]:
x_data=[]
y_data=[]
y_resized=y_resized.astype(float)
y_resized.shape
x_resized.shape
print(y_resized[0])
for i in range(len(y_resized)):
    for j in range(5):
        y_data.append(y_resized[i])
len(y_data)

[0.17 0.2  0.19 0.2  0.23]


4625

In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

for img in x_resized:


    datagen = ImageDataGenerator(
    rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


    img = img.reshape((1,) + img.shape)


    augmented_images = []
    num_augmented_images = 5  # Number of augmented images to generate

    for _ in range(num_augmented_images):
        for batch in datagen.flow(img, batch_size=1):
            augmented_images.append(batch[0].astype('uint8'))
            break
    for i in augmented_images:


        x_data.append(i)








In [24]:
print(len(x_data),"  ",len(y_data))
print(x_data[0].shape,"  ",y_data[0].shape)
x_data=np.array(x_data)
y_data=np.array(y_data)
print(x_data.shape,"  ",y_data.shape)

4625    4625
(224, 224, 3)    (5,)
(4625, 224, 224, 3)    (4625, 5)


In [25]:
from tensorflow.keras.callbacks import ReduceLROnPlateau


# Define the ReduceLROnPlateau callback
reduce_lr = ReduceLROnPlateau(factor=0.3, patience=5, verbose=1)

# Train the model with the callback
history = classifier1.fit(x_data, y_data, epochs=30, batch_size=64, callbacks=[reduce_lr], validation_split=0.2)


Epoch 1/30
58/58 [==============================] - 36s 618ms/step - loss: 0.0051 - mean_squared_error: 0.0051 - val_loss: 0.0033 - val_mean_squared_error: 0.0033 - lr: 1.0000e-04
Epoch 2/30
58/58 [==============================] - 23s 391ms/step - loss: 0.0045 - mean_squared_error: 0.0045 - val_loss: 0.0027 - val_mean_squared_error: 0.0027 - lr: 1.0000e-04
Epoch 3/30
58/58 [==============================] - 24s 413ms/step - loss: 0.0040 - mean_squared_error: 0.0040 - val_loss: 0.0023 - val_mean_squared_error: 0.0023 - lr: 1.0000e-04
Epoch 4/30
58/58 [==============================] - 23s 404ms/step - loss: 0.0036 - mean_squared_error: 0.0036 - val_loss: 0.0020 - val_mean_squared_error: 0.0020 - lr: 1.0000e-04
Epoch 5/30
58/58 [==============================] - 23s 402ms/step - loss: 0.0034 - mean_squared_error: 0.0034 - val_loss: 0.0018 - val_mean_squared_error: 0.0018 - lr: 1.0000e-04
Epoch 6/30
58/58 [==============================] - 24s 421ms/step - loss: 0.0031 - mean_squared_err

In [ ]:
# Save model architecture, weights, and optimizer state
classifier1.save('/content/drive/My Drive/final_year/profile_model_vgg_aug1.keras')

# Save training history
np.save('/content/drive/My Drive/final_year/profile_history_vgg_aug1.npy', history.history)
